In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv('../input/train.csv')
df[['Embarked']]=df[['Embarked']].fillna(df.mode().iloc[0])
x=df.iloc[:,[2,4,5,6,7,9,11]].values
y=df.iloc[:,1].values

dft=pd.read_csv('../input/test.csv')
dft[['Fare']]=dft[['Fare']].fillna(dft.median().iloc[0])
X_test=dft.iloc[:,[1,3,4,5,6,8,10]].values

from sklearn.preprocessing import Imputer
imputer=Imputer(missing_values='NaN',strategy='mean',axis=0)
imputer=imputer.fit(x[:,[2]])
x[:,[2]]=imputer.fit_transform(x[:,[2]])

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
lenc1=LabelEncoder()
x[:,1]=lenc1.fit_transform(x[:,1])
lenc2=LabelEncoder()
x[:,6]=lenc2.fit_transform(x[:,6])  

onehotenc=OneHotEncoder(categorical_features=[6],sparse=False)
x=onehotenc.fit_transform(x)

imputer=Imputer(missing_values='NaN',strategy='mean',axis=0)
imputer=imputer.fit(X_test[:,[2]])
X_test[:,[2]]=imputer.fit_transform(X_test[:,[2]])

X_test[:,1]=lenc1.transform(X_test[:,1])
X_test[:,6]=lenc2.transform(X_test[:,6])
X_test=onehotenc.transform(X_test)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)
"""
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
x_train=sc_x.fit_transform(x_train)
x_test=sc_x.transform(x_test)
X_test=sc_x.transform(X_test)"""

"""from sklearn.svm import SVC
classifier=SVC(kernel='rbf',random_state=0,gamma=0.1)
classifier.fit(x_train,y_train)
clf=SVC()"""

from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
classifier.fit(x_train,y_train)

y_pred=classifier.predict(x_test)


from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(classifier,X=x_train,y=y_train,cv=10)
accuracies.mean()
accuracies.std()

y_pred1=classifier.predict(X_test)

submission=pd.DataFrame({'PassengerId':dft.PassengerId.values,'Survived':y_pred1})
submission.to_csv('Submit1.csv',index=False)
